In [1]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode

In [2]:
#Criando arquivo com as disciplinas
dados = pd.read_csv('../dados/oferta_2025_1_depto-estatística---brasília_graduação.csv')
dados['disciplina'] = dados['disciplina'].apply(unidecode).str.upper().str.strip()
dados['tipo_disciplina'] = 'REGULAR'
dados.loc[dados['disciplina'].isin(['ESTATISTICA APLICADA', 'BIOESTATISTICA', 'PROBABILIDADE E ESTATISTICA']), 'tipo_disciplina'] = 'SERVICO'
dados['id_disciplina'] = dados.apply(lambda x: x['codigo_disciplina'] + '_' + str(x['codigo_turma']).zfill(2), axis =1)
dados[['id_disciplina', 'disciplina', 'carga_horaria', 'tipo_disciplina']].to_csv('../dados/disciplinas.csv', index=False)

In [3]:
#Criando matriz de conflitos
def parse_horario(horario_str):
    """
    Interpreta uma string de horário no formato '24M12' e a converte
    em um dicionário estruturado.

    Args:
        horario_str (str): A string de horário (ex: '35T34', '6N12', etc.).

    Returns:
        dict: Um dicionário com os dias, turno e horários, ou None se o formato for inválido.
    """
    # Expressão regular para capturar os 3 componentes: dias, turno, blocos
    match = re.match(r'(\d+)([MTN])(\d+)', horario_str)
    
    if not match:
        return None # Formato inválido

    dias_str, turno_char, blocos_str = match.groups()

    mapa_dias = {'2': 'SEG', '3': 'TER', '4': 'QUA', '5': 'QUI', '6': 'SEX'}
    
    # Converte os dígitos dos dias em um conjunto de strings (ex: {'SEG', 'QUA'})
    dias_set = {mapa_dias[dia] for dia in dias_str if dia in mapa_dias}
    
    # Converte os dígitos dos blocos em um conjunto de inteiros (ex: {1, 2})
    blocos_set = {int(bloco) for bloco in blocos_str}

    return {
        'dias': dias_set,
        'turno': turno_char,
        'blocos': blocos_set
    }

def verificar_conflito(horario_str1, horario_str2):
    """
    Verifica se duas strings de horário entram em conflito.

    Args:
        horario_str1 (str): Horário da primeira disciplina.
        horario_str2 (str): Horário da segunda disciplina.

    Returns:
        bool: True se houver conflito, False caso contrário.
    """
    h1 = parse_horario(horario_str1)
    h2 = parse_horario(horario_str2)

    if h1 is None or h2 is None:
        print(f"Aviso: Formato de horário inválido encontrado ({horario_str1} ou {horario_str2})")
        return False

    # 1. Verifica se os turnos são diferentes (se forem, não há conflito)
    if h1['turno'] != h2['turno']:
        return False

    # 2. Verifica se há intersecção de dias da semana
    dias_em_comum = h1['dias'].intersection(h2['dias'])
    if not dias_em_comum:
        return False

    # 3. Verifica se há intersecção de blocos de horário
    blocos_em_comum = h1['blocos'].intersection(h2['blocos'])
    if not blocos_em_comum:
        return False

    # Se passou por todas as verificações, significa que há conflito
    return True


disciplinas = dados['id_disciplina'].tolist()
horarios = dados.set_index('id_disciplina')['horario'].to_dict()
num_disciplinas = len(disciplinas)
matriz_conflitos = pd.DataFrame(0, index=disciplinas, columns=disciplinas)


for i in range(num_disciplinas):
    for j in range(i, num_disciplinas):
        disc1 = disciplinas[i]
        disc2 = disciplinas[j]
        
        # Não compara uma disciplina com ela mesma
        if i == j:
            continue
            
        if verificar_conflito(horarios[disc1], horarios[disc2]):
            matriz_conflitos.loc[disc1, disc2] = 1
            matriz_conflitos.loc[disc2, disc1] = 1
matriz_conflitos.to_csv('../dados/matriz_conflitos_disciplinas.csv', index=True)

In [4]:
#Criando arquivo com os professores
#prof1 = pd.read_csv('../dados/oferta_2025_1_depto-estatística---brasília_graduação.csv')
#prof2 = pd.read_csv('../dados/oferta_2025_2_depto-estatística---brasília_graduação.csv')
#prof = pd.concat([prof1, prof2])
prof = pd.read_csv('../dados/oferta_2025_1_depto-estatística---brasília_graduação.csv')

prof['docente'] = prof['docente'].str.upper().apply(unidecode)
prof = prof.groupby(['docente', 'ano_periodo']).sum()[['carga_horaria']].reset_index().groupby(['docente']).mean()[['carga_horaria']].reset_index()
prof['id_docente'] = prof.reset_index().apply(lambda x: 'docente' + '_' + str(x['index'] +1).zfill(2), axis=1)
prof['carga_maxima'] = 200
prof[['id_docente', 'docente', 'carga_maxima']].to_csv('../dados/docentes.csv', index=False)

In [14]:
# Dicionário para corrigir nomes de disciplinas inconsistentes
MAPA_DISCIPLINAS = {
    'DELINEAMENTO E ANALISE DE EXPERIMENTOS 1': 'DELINEAMENTO E ANALISE DE EXPERIMENTOS',
    'COMPUTACAO EM ESTATISTICA 2 - R': 'COMPUTACAO EM ESTATISTICA 2: R',
    'COMPUTACAO EM ESTATISTICA 2 - PYTHON': 'COMPUTACAO EM ESTATISTICA 2: PYTHON',
    'TEORIA DE RESPOSTA AO ITEM': 'TEORIA DA RESPOSTA AO ITEM'
}
# Dicionário para corrigir nomes de docentes inconsistentes
MAPA_DOCENTES = {
    'ALAN RICARDO RICARDO DA SILVA': 'ALAN RICARDO DA SILVA',
    'FELIPE SOUSA QUINTINO SOUSA QUINTINO': 'FELIPE SOUSA QUINTINO',
    'DEMERSON ANDRE POLLI ANDRE POLLI': 'DEMERSON ANDRE POLLI',
    'LUIZ FERNANDES CANCADO': 'ANDRE LUIZ FERNANDES CANCADO',
    'MONTEIRO DE CASTRO GOMES': 'EDUARDO MONTEIRO DE CASTRO GOMES'
}


# Mapeamento de texto para valor numérico (invertendo a lógica 1=alto, 3=baixo)
MAPA_VALOR_PREFERENCIA = {
    '1': 3,  # 'Tenho muito interesse'
    '2': 2, 
    '3': 1,  
    'EM PRINCIPIO, NAO TENHO INTERESSE.': 0
}

ADICOES_E_REGRAS = [
    # Regra para Leandro: 'Não tenho interesse' em tudo, exceto TCC 1
    {"docente": "LEANDRO TAVARES CORREIA", "disciplina": "ALL", "preferencia": "EM PRINCIPIO, NAO TENHO INTERESSE."},
    {"docente": "LEANDRO TAVARES CORREIA", "disciplina": "TRABALHO DE CONCLUSAO DE CURSO 1", "preferencia": "1"},
]

# Carrega a lista mestre de disciplinas
df_disciplinas = pd.read_csv('../dados/disciplinas.csv')

# Carrega e prepara a lista mestre de professores
# (Mantendo sua lógica original para criar o arquivo 'docentes.csv')
prof_oferta = pd.read_csv('../dados/oferta_2025_1_depto-estatística---brasília_graduação.csv')
prof_oferta['docente'] = prof_oferta['docente'].str.upper().apply(unidecode)
df_prof = prof_oferta.groupby(['docente', 'ano_periodo']).sum(numeric_only=True)[['carga_horaria']].reset_index().groupby(['docente']).mean(numeric_only=True)[['carga_horaria']].reset_index()
df_prof['id_docente'] = df_prof.reset_index().apply(lambda x: 'docente' + '_' + str(x['index'] + 1).zfill(2), axis=1)
df_prof['carga_maxima'] = 200
df_prof[['id_docente', 'docente', 'carga_maxima']].to_csv('../dados/docentes.csv', index=False)

# Carrega e limpa as respostas da pesquisa
df_respostas = pd.read_csv('../dados/RespostaOfertaProfessores.csv', encoding='latin1', sep='#')
df_respostas = df_respostas.melt(id_vars=['Nome', 'Horários de preferência'], var_name='disciplina', value_name='preferencia')[['Nome', 'disciplina', 'preferencia']].rename(columns={'Nome': 'docente'})
df_respostas['disciplina'] = df_respostas['disciplina'].str.upper().apply(unidecode).replace(MAPA_DISCIPLINAS)
df_respostas['docente'] = df_respostas['docente'].str.upper().apply(unidecode).replace(MAPA_DOCENTES)

# Caso especial para 'TÓPICOS' que se desdobram em duas disciplinas
df_topicos = df_respostas[df_respostas['disciplina'].str.contains('TOPICOS')].copy()
if not df_topicos.empty:
    df_topicos1 = df_topicos.copy()
    df_topicos1['disciplina'] = 'TOPICOS EM ESTATISTICA 1'
    df_topicos2 = df_topicos.copy()
    df_topicos2['disciplina'] = 'TOPICOS EM ESTATISTICA 2'
    df_respostas = pd.concat([df_respostas[~df_respostas['disciplina'].str.contains('TOPICOS')], df_topicos1, df_topicos2])

# --- 3. CONSTRUÇÃO DA MATRIZ COMPLETA DE PREFERÊNCIAS ---

# Cria um "template" com todas as combinações de (docente, disciplina)
todos_docentes = df_prof[['id_docente', 'docente']]
todas_disciplinas = df_disciplinas[['id_disciplina', 'disciplina']]
df_base = todos_docentes.assign(key=1).merge(todas_disciplinas.assign(key=1), on='key').drop('key', axis=1)

# Junta as respostas da pesquisa com o template. Onde não houve resposta, o valor será NaN.
df_final = pd.merge(df_base, df_respostas, on=['docente', 'disciplina'], how='left')

# --- 4. APLICAÇÃO DAS REGRAS E ADIÇÕES PONTUAIS ---

# Define a coluna 'preferencia' como o índice para facilitar as buscas
df_final.set_index(['docente', 'disciplina'], inplace=True)

# Aplica as regras da lista ADICOES_E_REGRAS
for regra in ADICOES_E_REGRAS:
    docente = regra['docente']
    disciplina = regra['disciplina']
    preferencia = regra['preferencia']
    
    # Seleciona as linhas a serem alteradas
    if docente == 'ALL':
        idx_docente = slice(None) # Todos os docentes
    else:
        idx_docente = docente
        
    if disciplina == 'ALL':
        idx_disciplina = slice(None) # Todas as disciplinas
    else:
        idx_disciplina = disciplina
        
    # Atualiza os valores na coluna 'preferencia'
    df_final.loc[(idx_docente, idx_disciplina), 'preferencia'] = preferencia

# Reseta o índice para o formato original
df_final.reset_index(inplace=True)

# --- 5. FINALIZAÇÃO E EXPORTAÇÃO (LÓGICA ALTERADA) ---

# Identifica os grupos para a lógica condicional
professores_que_responderam = set(df_respostas['docente'].unique())
docentes_nao_responderam = set(df_prof['docente']) - professores_que_responderam
disciplinas_servico = set(df_disciplinas[df_disciplinas['tipo_disciplina'] == 'SERVICO']['disciplina'])

# Cria as máscaras booleanas para aplicar as regras
mascara_nula = df_final['preferencia'].isnull()
mascara_nao_respondente = df_final['docente'].isin(docentes_nao_responderam)
mascara_servico = df_final['disciplina'].isin(disciplinas_servico)

# Aplica a regra para NÃO-RESPONDENTES em DISCIPLINAS DE SERVIÇO
df_final.loc[mascara_nula & mascara_nao_respondente & mascara_servico, 'preferencia'] = '1'

# Preenche qualquer outro valor nulo (ex: quem respondeu mas pulou algo) com a preferência baixa
df_final['preferencia'].fillna('EM PRINCIPIO, NAO TENHO INTERESSE.', inplace=True)

# Mapeia os valores de texto para numéricos
df_final['preferencia'] = df_final['preferencia'].astype(str).str.upper().apply(unidecode).map(MAPA_VALOR_PREFERENCIA)

# Seleciona e salva as colunas finais
df_final[['id_docente', 'id_disciplina', 'preferencia']].to_csv('../dados/preferencias.csv', index=False)


C:\Users\kmenezes\AppData\Local\Temp\ipykernel_21052\3879714059.py:92: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_final.loc[(idx_docente, idx_disciplina), 'preferencia'] = preferencia
C:\Users\kmenezes\AppData\Local\Temp\ipykernel_21052\3879714059.py:113: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final['preferencia'].fillna('EM PRINCIPIO, NAO TENHO INTERESSE.', inplace=True)


In [7]:
dados.merge(prof, on = 'docente',how='left',indicator=True).merge(df_preferencias, on = ['id_disciplina', 'id_docente'],how='left')[['id_disciplina', 'id_docente', 'preferencia']].fillna(1).to_csv('../dados/solucao_25_1.csv',index=False)

In [8]:
disciplinas = pd.read_csv('../dados/disciplinas.csv')
preferencias = pd.read_csv('../dados/preferencias.csv')
docentes = pd.read_csv('../dados/docentes.csv')
s1 = pd.read_csv('../dados/oferta_2025_1_depto-estatística---brasília_graduação.csv')
s2 = pd.read_csv('../dados/oferta_2025_2_depto-estatística---brasília_graduação.csv')


s1['disciplina'] = s1['disciplina'].str.upper().apply(unidecode)
s2['disciplina'] = s2['disciplina'].str.upper().apply(unidecode)

In [9]:
s1.merge(disciplinas, on ='disciplina', how='left')

,codigo_disciplina,disciplina,codigo_turma,ano_periodo,docente,carga_horaria_x,horario,horario_extenso,vagas_ofertadas,vagas_ocupadas,local,id_disciplina,carga_horaria_y,tipo_disciplina
0,EST0001,COMPUTACAO EM ESTATISTICA 1,1,2025.1,JAMES MATOS SAMPAIO,30,6M34,Sexta-feira 10:00 às 11:50,NaN,38,37,EST0001_01,30,REGULAR
1,EST0001,COMPUTACAO EM ESTATISTICA 1,1,2025.1,JAMES MATOS SAMPAIO,30,6M34,Sexta-feira 10:00 às 11:50,NaN,38,37,EST0001_02,30,REGULAR
2,EST0001,COMPUTACAO EM ESTATISTICA 1,2,2025.1,JAMES MATOS SAMPAIO,30,3T45,Terça-feira 16:00 às 17:50,NaN,35,31,EST0001_01,30,REGULAR
3,EST0001,COMPUTACAO EM ESTATISTICA 1,2,2025.1,JAMES MATOS SAMPAIO,30,3T45,Terça-feira 16:00 às 17:50,NaN,35,31,EST0001_02,30,REGULAR
4,EST0004,ESTATISTICA COMPUTACIONAL,1,2025.1,HELTON SAULO BEZERRA DOS SANTOS,60,3T2345,Terça-feira 14:00 às 17:50,NaN,40,23,EST0004_01,60,REGULAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,EST0086,ANALISE DE SOBREVIVENCIA,1,2025.1,JULIANA BETINI FACHINI GOMES,60,24T45,Segunda-feira 16:00 às 17:50Quarta-feira 16:00...,NaN,40,20,EST0086_01,60,REGULAR
249,EST0088,PRATICAS DE EXTENSAO EM ESTATISTICA 2,1,2025.1,LUCAS MOREIRA,60,24T45,Segunda-feira 16:00 às 17:50Quarta-feira 16:00...,NaN,40,17,EST0088_01,60,REGULAR
250,EST0091,COMPUTACAO EM ESTATISTICA 2: R,1,2025.1,DONALD MATTHEW PIANTO,60,35M34,Terça-feira 10:00 às 11:50Quinta-feira 10:00 à...,NaN,37,24,EST0091_01,60,REGULAR
251,EST0092,COMPUTACAO EM ESTATISTICA 2: PYTHON,1,2025.1,JOSE AUGUSTO FIORUCCI,60,24T23,Segunda-feira 14:00 às 15:50Quarta-feira 14:00...,NaN,37,24,EST0092_01,60,REGULAR


In [10]:
disciplinas

,id_disciplina,disciplina,carga_horaria,tipo_disciplina
0,EST0001_01,COMPUTACAO EM ESTATISTICA 1,30,REGULAR
1,EST0001_02,COMPUTACAO EM ESTATISTICA 1,30,REGULAR
2,EST0004_01,ESTATISTICA COMPUTACIONAL,60,REGULAR
3,EST0005_01,INFERENCIA BAYESIANA,60,REGULAR
4,EST0011_01,MODELOS LINEARES GENERALIZADOS,60,REGULAR
5,EST0017_01,METODOS ESTATISTICOS 2,60,REGULAR
6,EST0019_01,ESTATISTICA APLICADA,90,SERVICO
7,EST0019_02,ESTATISTICA APLICADA,90,SERVICO
8,EST0019_03,ESTATISTICA APLICADA,90,SERVICO
9,EST0019_04,ESTATISTICA APLICADA,90,SERVICO
